![](http://)

# **Introduction**

# I will analyze the financial data of Roast coffee shop and answer some questions.

* How do your actual sales and expenses differ from your forecasts? What causes these differences?

* Are there certain months or seasons when your sales peak or decline? What causes these trends?
* What are your most and least profitable months?

* How are your expenses trending throughout the year?
* Are there any months or seasons that you need to address?
* Which expense categories have the biggest variance from your forecasts?

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/roast-coffee-shop-2023-financials/Coffee_Shop_Financial_Data_2023.csv


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("/kaggle/input/roast-coffee-shop-2023-financials/Coffee_Shop_Financial_Data_2023.csv")
df.head()

,Date,Category,Forecasted Amount,Actual Amount
0,2023-01-01,Coffee Sales,980,954
1,2023-01-01,Food Sales,646,703
2,2023-01-01,Ingredients Cost,457,447
3,2023-01-01,Labor,182,188
4,2023-01-01,Rent,384,375


Identify and calculate the percentage of the missing values in each attribute.

In [20]:
df.describe(include='all')

,Date,Category,Forecasted Amount,Actual Amount
count,2555,2555,2555.000000,2555.000000
unique,NaN,7,NaN,NaN
top,NaN,Coffee Sales,NaN,NaN
freq,NaN,365,NaN,NaN
mean,2023-07-02 00:00:00,NaN,382.334247,381.252446
min,2023-01-01 00:00:00,NaN,100.000000,90.000000
25%,2023-04-02 00:00:00,NaN,230.000000,229.000000
50%,2023-07-02 00:00:00,NaN,344.000000,343.000000
75%,2023-10-01 00:00:00,NaN,466.500000,463.000000
max,2023-12-31 00:00:00,NaN,999.000000,1087.000000


In [4]:
df.isnull()

,Date,Category,Forecasted Amount,Actual Amount
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
2550,False,False,False,False
2551,False,False,False,False
2552,False,False,False,False
2553,False,False,False,False


In [6]:
df.dtypes

Date                 object
Category             object
Forecasted Amount     int64
Actual Amount         int64
dtype: object

In [7]:
df['Category'].value_counts()

Category
Coffee Sales        365
Food Sales          365
Ingredients Cost    365
Labor               365
Rent                365
Utilities           365
Marketing           365
Name: count, dtype: int64

**Data Visualization**

In [34]:
df['Month'] = df['Date'].dt.to_period('M')

# Displaying totals by category by month using pivot table
monthly_df = df.pivot_table(index='Month', columns='Category', 
                            values=['Forecasted Amount', 'Actual Amount'], 
                            aggfunc='sum')

print(monthly_df)

monthly_df.to_csv('monthly_summary.csv', index=True)
print('The csv file has been saved.')

         Actual Amount                                                     \
Category  Coffee Sales Food Sales Ingredients Cost  Labor Marketing  Rent   
Month                                                                       
2023-01          19461      17483             9198   9467      9817  8504   
2023-02          16981      17792             8014   8409      9429  8276   
2023-03          19137      19194             9093   9674      8430  9419   
2023-04          18939      18589             8265   7143      8126  9093   
2023-05          18176      19604             9043   9266      9028  8722   
2023-06          15142      17249             8752   9060      9032  9229   
2023-07          20023      18035             9485  10572      9802  9782   
2023-08          19033      19161             9536   9044      9630  9352   
2023-09          16843      17014             9399   9790      9642  9357   
2023-10          19605      17418             8881   9651      8969  8313   

In [41]:
# Combine coffee sales with food sales
monthly_df[('Actual Amount', 'Total Sales')] = (
    monthly_df[('Actual Amount', 'Coffee Sales')] + 
    monthly_df[('Actual Amount', 'Food Sales')]
)

monthly_df[('Forecasted Amount', 'Total Sales')] = (
    monthly_df[('Forecasted Amount', 'Coffee Sales')] + 
    monthly_df[('Forecasted Amount', 'Food Sales')]
)

monthly_df[('Actual Amount', 'Total Cost')]=(
    monthly_df[('Actual Amount', 'Ingredients Cost')] + monthly_df[('Actual Amount','Labor')] +
    monthly_df[('Actual Amount', 'Rent')]+ monthly_df[('Actual Amount','Utilities')]+
    monthly_df[('Actual Amount', 'Marketing')]
)
monthly_df[('Forecasted Amount', 'Total Cost')] = (
    monthly_df[('Forecasted Amount', 'Ingredients Cost')] + monthly_df[('Forecasted Amount', 'Labor')] +
    monthly_df[('Forecasted Amount', 'Rent')] + monthly_df[('Actual Amount', 'Utilities')]+
    monthly_df[('Forecasted Amount', 'Rent')]
)

# Show totals
print(monthly_df[[('Actual Amount', 'Total Sales'), ('Forecasted Amount', 'Total Sales')]])
print(monthly_df[[('Actual Amount', 'Total Cost'), ('Forecasted Amount', 'Total Cost')]])

         Actual Amount Forecasted Amount
Category   Total Sales       Total Sales
Month                                   
2023-01          36944             37232
2023-02          34773             34607
2023-03          38331             37988
2023-04          37528             37644
2023-05          37780             38178
2023-06          32391             32887
2023-07          38058             37879
2023-08          38194             38109
2023-09          33857             34090
2023-10          37023             37221
2023-11          33471             33907
2023-12          34670             34838
         Actual Amount Forecasted Amount
Category    Total Cost        Total Cost
Month                                   
2023-01          45523             44403
2023-02          42598             41761
2023-03          45524             46207
2023-04          41155             42403
2023-05          44923             44727
2023-06          45399             45965
2023-07         

In [54]:
monthly_df.to_csv('monthly_Sales_Cost.csv', index=True)
print("The csv file has been saved.")

The csv file has been saved.


# Monthly actual vs. forecast sales variance

In [61]:
import plotly.graph_objects as go

#Convert month index to string
months = monthly_df.index.astype(str)
# Creating a Sales Graph
fig_sales = go.Figure()

# add the Actual Sales
fig_sales.add_trace(go.Bar(
    x=months,
    y=actual_sales,
    name='Actual Sales',
    marker_color='blue'
))

# add the Forecasted Sales
fig_sales.add_trace(go.Bar(
    x=months,
    y=forecasted_sales,
    name='Forecasted Sales',
    marker_color='lightblue'
))

# graph layout
fig_sales.update_layout(
    title='Monthly Sales: Actual vs Forecasted',
    xaxis_title='Months',
    yaxis_title='Sales Amount',
    barmode='group',
    width=750
)

# show graph
fig_sales.show()

# create cost graph
fig_costs = go.Figure()

# add the Actual Costs
fig_costs.add_trace(go.Bar(
    x=months,
    y=actual_costs,
    name='Actual Costs',
    marker_color='orange'
))

# add the Forecasted Costs
fig_costs.add_trace(go.Bar(
    x=months,
    y=forecasted_costs,
    name='Forecasted Costs',
    marker_color='lightgoldenrodyellow'
))

# graph layout
fig_costs.update_layout(
    title='Monthly Costs: Actual vs Forecasted',
    xaxis_title='Months',
    yaxis_title='Cost Amount',
    barmode='group',
    width=750
)

# show graph
fig_costs.show()


**The months in which actual sales were lower than expected** were
January, April, May, June, September, October, November and December.
However, the differences were minimal.

* Sales peak in March and August, with a notable drop in June.


**The months in which the actual costs were higher than the forecast** were
January, February, May, September, October, and December.
The difference was also small, so the forecast was good.

* In terms of expenses, July is particularly high and February and April are low.
* Even though sales are down in June, expenses are not that different from other months, so action is needed.

In [71]:
import plotly.graph_objects as go
import plotly.graph_objects as go

# Check the headers of the data frame (it's hierarchical, so you'll need to manipulate the column names)
print(monthly_df.columns)

# Obtaining predicted and actual values ​​by category
categories = monthly_df.columns.levels[1]  # name of category
months = monthly_df.index.astype(str)  # Monthly Data

# カテゴリごとに異なる色を指定
category_colors = {
    'Coffee Sales': {'forecasted': '#D2B48C', 'actual': '#A52A2A'},  # lightbrown → #D2B48C, brown → #A52A2A
    'Food Sales': {'forecasted': '#90EE90', 'actual': '#008000'},    # lightgreen → #90EE90, green → #008000
    'Ingredients Cost': {'forecasted': '#ADD8E6', 'actual': '#0000FF'},  # lightblue → #ADD8E6, blue → #0000FF
    'Labor': {'forecasted': '#FFB6C1', 'actual': '#FF0000'},         # lightpink → #FFB6C1, red → #FF0000
    'Rent': {'forecasted': '#FFFFE0', 'actual': '#FFA500'},          # lightyellow → #FFFFE0, orange → #FFA500
    'Utilities': {'forecasted': '#D3D3D3', 'actual': '#808080'},     # lightgray → #D3D3D3, gray → #808080
    'Marketing': {'forecasted': '#00FFFF', 'actual': '#008B8B'},     # lightpurple → #E6E6FA, purple → #800080
    'Total Sales': {'forecasted': '#FFD700', 'actual': '#DAA520'},
    'Total Cost': {'forecasted': '#87CEEB', 'actual': '#4682B4'} 
}

for category in categories:
    # Predicted and actual values ​​for each category
    forecasted = monthly_df['Forecasted Amount'][category]
    actual = monthly_df['Actual Amount'][category]
    
    # create graph
    fig = go.Figure()

    # Added achievement bars
    fig.add_trace(go.Bar(
        x=months,
        y=actual,
        name=f'Actual {category}',
        marker_color=category_colors[category]['actual']
    ))

    # Added forecast bars 
    fig.add_trace(go.Bar(
        x=months,
        y=forecasted,
        name=f'Forecasted {category}',
        marker_color=category_colors[category]['forecasted']
    ))

    # layout
    fig.update_layout(
        title=f'Monthly Comparison for {category}',
        xaxis_title='Month',
        yaxis_title='Amount',
        barmode='group',
        width=800  # width
    )

    # show graph
    fig.show()



MultiIndex([(    'Actual Amount',     'Coffee Sales'),
            (    'Actual Amount',       'Food Sales'),
            (    'Actual Amount', 'Ingredients Cost'),
            (    'Actual Amount',            'Labor'),
            (    'Actual Amount',        'Marketing'),
            (    'Actual Amount',             'Rent'),
            (    'Actual Amount',        'Utilities'),
            ('Forecasted Amount',     'Coffee Sales'),
            ('Forecasted Amount',       'Food Sales'),
            ('Forecasted Amount', 'Ingredients Cost'),
            ('Forecasted Amount',            'Labor'),
            ('Forecasted Amount',        'Marketing'),
            ('Forecasted Amount',             'Rent'),
            ('Forecasted Amount',        'Utilities'),
            (    'Actual Amount',      'Total Sales'),
            ('Forecasted Amount',      'Total Sales'),
            (    'Actual Amount',       'Total Cost'),
            ('Forecasted Amount',       'Total Cost')],
         

# Overall, there is almost no difference between the actual results and the forecasts, so we can say that the accounting department is functioning very well.

If I had to say something, it would be that the decrease in coffee sales in June was particularly noticeable, so we need to investigate in more detail what is going on.